In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import time

# To get Specializations in Coursera

In [ ]:
#For Specialization
spl_title=[]
spl_link=[]

import timeit

import time
start = time.time()
for i in range(1,16):
    #44&index=prod_all_products_term_optimization_v3&entityTypeDescription=Specializations&allLanguages=English
    url = "https://www.coursera.org/directory/specializations?page=" +str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    st = soup.find_all('a', class_ ="c-directory-link")
    num=len(st)
    for j in range(1,num):
        #spl-title
        st = soup.find_all('a', class_ ="c-directory-link")[j].get_text()
        spl_title.append(st)       
        #spl URL
        curl= soup.find_all("a", class_='c-directory-link')[j]
        spl_link.append("https://www.coursera.org"+ curl.get('href'))
stop = time.time()
print(f"Web Scraping time: {stop - start}s")
#creating a dataframe of all specializations
splz=pd.DataFrame({'spl_title':spl_title,
                   'spl_link':spl_link})


#writing into a .csv file
#coursera_df.to_csv('Coursera_catalog.csv')

Training time: 26.182609796524048s


# To get courses in Coursera

In [ ]:
#get all courses on coursera
course_title=[]
course_link=[]

cnt=0
for i in range(1,153):
    #44&index=prod_all_products_term_optimization_v3&entityTypeDescription=Specializations&allLanguages=English
    url = "https://www.coursera.org/directory/courses?page=" +str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    st = soup.find_all('a', class_ ="c-directory-link")
    num=len(st)
    for j in range(1,num):
        #course-title
        ct = soup.find_all('a', class_ ="c-directory-link")[j].get_text()
        cnt=cnt+1
        course_title.append(ct)       
        #course URL
        curl= soup.find_all("a", class_='c-directory-link')[j]
        course_link.append("https://www.coursera.org"+ curl.get('href'))

#creating a dataframe of all specializations
courses=pd.DataFrame({'course_title':course_title,
                   'course_link':course_link})

#writing into a .csv file
#coursera_df.to_csv('Coursera_catalog.csv')

# To Get Course Details of each Course

In [ ]:
#To get detail of each course

course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
course_tit=[]
time_required=[]
course_level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
course_rating=[]
start = time.time()
for urlC,pg in zip(courses.course_link,courses.index):
    page_course = requests.get(urlC)
    soup = BeautifulSoup(page_course.content, 'html.parser')
    
    guided=soup.find('div',class_="_fc5ifbq _bv93ce")
    
    if guided is None:
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="banner-title m-b-0 banner-title-without--subtitle")
      if cnn is None:
        continue
      else:
        cnn=cnn.get_text()
        course_name.append(cnn)

      #Attaching Course Link
      course_link.append(urlC)
      #Attaching type of course
      typ.append('COURSE')
      
      #attaching University /name
      univ = soup.find('h4', class_ = 'headline-4-text bold rc-Partner__title').get_text()
      university_name.append(univ)

      #attaching University photo
      try:

        u_logo=soup.find('div',class_='rc-Partner')
        l=u_logo.find('img').get('src')
        university_logo.append(l)
      except:
        university_logo.append("not-mentioned")
      
      #Attaching Course Level:
      try:

        fet=soup.find_all('div',class_='_16ni8zai m-b-0 m-t-1s')
        fett=soup.find_all('div',class_='_16ni8zai m-b-0')
      
        level=''
        for i in fet:
          if i is None:
            continue
          elif "Level" in i.get_text():
            level=i.get_text()
            course_level.append(level)
            break
          else:
            continue
        if level is '':
          for i in fett:
            if i is None:
              continue
            elif "Level" in i.get_text():
              level=i.get_text()
              course_level.append(level)
              break
            else:
              continue
  
        if level is '':
          course_level.append("not-mentioned")
      except:
        course_level.append("not-mentioned")

        

      #attaching time required to complete the course
      try:

        fet=soup.find_all('div',class_='_16ni8zai m-b-0 m-t-1s')
      
        com=''
        for i in fet:
          if i is None:
            continue
          elif "complete" in i.get_text():
            com=i.get_text()
            time_required.append(com)
            break
          else:
            continue
  
        if com is '':
          time_required.append("not-mentioned")
      except:
        time_required.append("not-mentioned")
  
      #Attaching Course Languages and course subtitles
      try:

        
        lang=soup.find_all("div",class_="_y1d9czk m-b-2 p-t-1s")[-1]
        lgg= lang.find("div",class_="_16ni8zai m-b-0").get_text()
        subtitless=lang.find("div",class_="content-inner").get_text()
        course_language.append(lgg)
        course_subtitles.append(subtitless)
      except:
        course_language.append("not-mentioned")
        course_subtitles.append("not-mentioned")

      #Attachng course skills
      try:

        skills= [x.get_text() for x in soup.find_all('span', class_ = '_rsc0bd m-r-1s m-b-1s')]
        course_skills.append(skills)
        skills=[]
      except:
        course_skills.append("not-mentioned")

      # Attaching Course Rating
      try:
        rate=soup.find('span',class_='_16ni8zai m-b-0 rating-text number-rating number-rating-expertise').get_text()
        course_rating.append(rate)
      except:
        course_rating.append("not-mentioned")
      
      #attaching Course Category and sub-category
      try:
        subj = soup.find_all('a',class_ = '_172v19u6 color-white font-weight-bold')
        category.append(subj[1].get_text())
        sub_category.append(subj[2].get_text())
      except IndexError:
        category.append("not-mentioned")
        sub_category.append("not-mentioned")
      print(pg)
    else:
      #Attaching Course Link
      course_link.append(urlC)
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="_125g251l _gkjc69").get_text()
      course_name.append(cnn)
      #Attaching type of course
      typ.append('GUIDED PROJECT')
      
      #attaching University /name
      university_name.append('Coursera Project Network')

      #attaching University photo
      u_logo=soup.find('div',class_='_1tu07i3a')
      l=u_logo.find('img').get('src')
      university_logo.append(l)
      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
      try:
        lev=soup.find_all("div",class_="_8m7gipb _1f096on")[1]
        level= lang.find("span").get_text()
      
        course_level.append(level)
      
      #time required to complete the course
        vtim=soup.find('span',class_='_1rcyblj').get_text()
        time_required.append(vtim)
      #Attaching Course Languages and course subtitles
        lang=soup.find_all("span",class_="_1rcyblj")[-2].get_text()
        course_language.append(lang)
        course_subtitles.append(lang)

      except:
        course_level.append("mixed/not-mentioned")
        time_required.append("not-mentioned")
        course_language.append("not-mentioned")
        course_subtitles.append("not-mentioned")


      #Attachng course skills
      try:
        skills= [x.get_text() for x in soup.find_all('span', class_ = '_1q9sh65')]
        course_skills.append(skills)
        skills=[]
      except:
        course_skills.append("not-mentioned")

      # Attaching Course Rating
      try:
        rate=soup.find('span',class_='_16ni8zai m-b-0 rating-text number-rating m-l-1s m-r-1').get_text()
        course_rating.append(rate)
      except:
        course_rating.append("not-mentioned")  

      #attaching Course Category and sub-category
      try:

        subj = soup.find_all('a',class_ = '_k8ap7s6')
        category.append(subj[1].get_text())
        sub_category.append(subj[2].get_text())
      except IndexError:       
        category.append("not-mentioned")
        sub_category.append("not-mentioned")
      print(pg)
stop = time.time()
#print(f"Web Scraping time: {stop - start}s")

#Attaching all extracted Course Details to Dataframe:
 Coursera_courses_catalog

In [ ]:
Coursera_courses_catalog=pd.DataFrame({'course_name':course_name,
                              'course_link':course_link,
                              'university_name':university_name,
                              'course_type':typ,
                              'university_logo':university_logo,
                              'time_required':time_required,
                              'course_language':course_language,
                              'course_subtitles':course_subtitles,
                              'course_skills':course_skills,
                              'course_rating':course_rating,
                              'category':category,
                              'sub_category':sub_category
                              'course_level':course_level})

In [8]:
Coursera_courses_catalog

,course_name,course_link,university_name,course_type,university_logo,time_required,course_language,course_subtitles,course_skills,course_rating,category,sub_category,course_level
0,Atención prehospitalaria del ictus agudo y se...,https://www.coursera.org/learn/ictus-agudo-esc...,Universitat de Barcelona,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 14 hours to complete,Spanish,Subtitles: Spanish,[],not-mentioned,Health,Patient Care,Intermediate Level
1,Brand & Content Marketing,https://www.coursera.org/learn/brand-and-conte...,IE Business School,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 15 hours to complete,English,"Subtitles: French, Portuguese (Brazilian), Rus...",[],4.2stars,Arts and Humanities,History,Intermediate Level
2,Cluster Analysis using RCmdr,https://www.coursera.org/learn/cluster-analysi...,Coursera Project Network,GUIDED PROJECT,https://d3njjcbhbojbot.cloudfront.net/api/util...,one hour,English,English,"['Cluster Analysis', 'Dendrogram', 'K-Means Cl...",not-mentioned,Data Science,Data Analysis,"Subtitles: French, Portuguese (Brazilian), Rus..."
3,"Comercio, Inmigración y Tipos de Cambio en un...",https://www.coursera.org/learn/comercio-inmigr...,IE Business School,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 11 hours to complete,Spanish,Subtitles: Spanish,[],not-mentioned,Social Sciences,Economics,Beginner Level
4,Create a Budget with Google Sheets,https://www.coursera.org/learn/create-a-budget...,Coursera Project Network,GUIDED PROJECT,https://d3njjcbhbojbot.cloudfront.net/api/util...,1 hour,English,English,"['Spreadsheet', 'Budget', 'Google Sheets', 'Te...",4.8stars,Business,Finance,Subtitles: Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5159,系统平台与计算环境,https://www.coursera.org/learn/jisuanji-wangluo,Xi'an Jiaotong University,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 12 hours to complete,Chinese (Simplified),Subtitles: Chinese (Simplified),[],not-mentioned,Computer Science,Software Development,not-mentioned
5160,紅樓夢(The Red Chamber Dream),https://www.coursera.org/learn/the-red-chamber...,National Taiwan University,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 19 hours to complete,Chinese (Traditional),"Subtitles: Chinese (Traditional), Chinese (Sim...",[],not-mentioned,Arts and Humanities,Philosophy,not-mentioned
5161,紅樓夢-母神崇拜 (The Red Chamber Dream (2)),https://www.coursera.org/learn/rcd2,National Taiwan University,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 26 hours to complete,Chinese (Traditional),"Subtitles: Chinese (Traditional), Chinese (Sim...",[],not-mentioned,Arts and Humanities,Philosophy,Beginner Level
5162,經濟學概論：誘因與市場（Introduction to Economics: Incenti...,https://www.coursera.org/learn/economics,National Taiwan University,COURSE,https://d3njjcbhbojbot.cloudfront.net/api/util...,Approx. 10 hours to complete,Chinese (Traditional),Subtitles: Chinese (Traditional),[],4.8stars,Social Sciences,Economics,not-mentioned


In [10]:
#writing into a .csv file
Coursera_courses_catalog.to_csv('Coursera_courses_catalog.csv')